# 01 Import libraries

In [1]:
import numpy as np
import torch

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using device: {device}")

Using device: cpu


In [2]:
import pandas as pd

In [3]:
import importlib

In [4]:
%reload_ext autoreload
%autoreload 2

In [5]:
import lib
importlib.reload(lib)

<module 'lib' from 'c:\\Users\\dongq\\OneDrive\\Desktop\\New results\\Portfolio Allocation\\Learning-the-Optimal-Solution-Path\\lib\\__init__.py'>

In [6]:
from lib.lsp.basis_generator import bivariate_legendre
from lib.lsp.basis_tf_module import Basis_TF_SGD
from lib.lsp.learn_solution_path import adaptive_lsp
from lib.lsp.loss_fn_lsp import allocation_cost
from lib.lsp.utils_lsp import get_sup_error_lsp_2d
from lib.fast_tensor_data_loader import FastTensorDataLoader

ImportError: cannot import name 'lsp_boosting' from 'lib.lsp.learn_solution_path' (c:\Users\dongq\OneDrive\Desktop\New results\Portfolio Allocation\Learning-the-Optimal-Solution-Path\lib\lsp\learn_solution_path.py)

# 02 Load data

In [ ]:
# file path for Colab. May need to change this
# X_df = pd.read_csv('/content/Learning-the-Optimal-Solution-Path/experiments/fair-regression/data/X_processed.csv')
# y_df = pd.read_csv('/content/Learning-the-Optimal-Solution-Path/experiments/fair-regression/data/y_processed.csv')
decomp_cov_df = pd.read_csv('decomp_cov.csv')
mean_df = pd.read_csv('mean.csv')

In [ ]:
decomp_cov = np.array(decomp_cov_df)
mean = np.array(mean_df).squeeze()

In [ ]:
decomp_cov = torch.tensor(decomp_cov, dtype=torch.float32)
mean = torch.tensor(mean, dtype=torch.float32)

In [ ]:
# full gradient descent uses all data points
GD_data_loader = FastTensorDataLoader(decomp_cov, mean, batch_size=len(decomp_cov), shuffle=False, )
# test data
test_data_loader = FastTensorDataLoader(decomp_cov, mean, batch_size=len(decomp_cov), shuffle=False, )

In [ ]:
lam_max_2d = [1, 1]
lam_min_2d = [0, 0]
input_dim = decomp_cov.shape[1] - 1

In [ ]:
# Read the CSV file into a DataFrame
# truth = pd.read_csv('/content/Learning-the-Optimal-Solution-Path/experiments/fair-regression/results/exact_soln_list.csv')
truth = pd.read_csv('exact_soln_list_2d.csv')

# Display the DataFrame
truth

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,15.195165,11.914677,11.955276,11.995770,12.036159,12.076448,12.116634,12.156718,12.196705,12.236592,12.276388,12.316086,12.355691,12.395209,12.434635,12.473982,12.513460,12.559717,12.605769,12.652280
1,14.411641,11.323114,11.363797,11.404370,11.444837,11.485199,11.525454,11.565603,11.605650,11.645598,11.685446,11.725196,11.764850,11.804410,11.843878,11.883253,11.922541,11.961743,12.000860,12.039893
2,13.628118,10.730240,10.771020,10.811688,10.852239,10.892681,10.933014,10.973238,11.013354,11.053367,11.093277,11.133083,11.172795,11.212406,11.251925,11.291348,11.330680,11.369923,11.409075,11.448140
3,12.844594,10.135834,10.176719,10.217484,10.258136,10.298669,10.339087,10.379393,10.419589,10.459677,10.499657,10.539532,10.579308,10.618981,10.658557,10.698036,10.737419,10.776708,10.815906,10.855016
4,12.061070,9.539598,9.580598,9.621478,9.662234,9.702869,9.743388,9.783785,9.824072,9.864246,9.904308,9.944264,9.984114,10.023858,10.063503,10.103045,10.142489,10.181840,10.221090,10.260247
5,11.277547,8.941166,8.982296,9.023297,9.064174,9.104922,9.145550,9.186058,9.226444,9.266717,9.306875,9.346920,9.386857,9.426688,9.466413,9.506031,9.545547,9.584966,9.624282,9.663498
6,10.494023,8.340089,8.381352,8.422487,8.463491,8.504368,8.545116,8.585744,8.626248,8.666632,8.706898,8.747054,8.787091,8.827019,8.866837,8.906548,8.946154,8.985652,9.025046,9.064337
7,9.710500,7.735779,7.777187,7.818461,7.859601,7.900612,7.941496,7.982255,8.022887,8.063400,8.103793,8.144068,8.184226,8.224271,8.264201,8.304020,8.343726,8.383324,8.422812,8.462193
8,8.926976,7.127531,7.169084,7.210499,7.251784,7.292938,7.333964,7.374864,7.415640,7.456291,7.496825,7.537237,7.577532,7.617707,7.657767,7.697713,7.737544,7.777260,7.816865,7.856357
9,8.143453,6.514608,6.556315,6.597885,6.639320,6.680623,6.721797,6.762846,6.803771,6.844573,6.885253,6.925814,6.966257,7.006580,7.046786,7.086874,7.126845,7.166700,7.206441,7.246068


In [ ]:
true_losses = truth.to_numpy()

In [ ]:
loss_fn = allocation_cost

# 03 Adaptively learn solution path

In [17]:
max_epochs = 300

In [18]:
phi_lam = bivariate_legendre

In [19]:
start_basis_dim = 5
end_basis_dim = 13
init_lr = .5**9

In [ ]:
def thresh_basis(basis_dim):
    return 0.1**(basis_dim/5 + 2.5)

In [ ]:
np.random.seed(8675309)
torch.manual_seed(8675309)

num_itr_history, sup_err_history, breaks = lsp_boosting(input_dim, start_basis_dim, end_basis_dim,
                                        phi_lam, max_epochs, GD_data_loader,  test_data_loader,
                                        loss_fn, lam_min_2d, lam_max_2d, true_losses, init_lr=init_lr,
                                        intercept=False, thresh_basis=thresh_basis, record_frequency=10,
                                        record_fctn=get_sup_error_lsp_2d,
                                        distribution='uniform', device=device, trace_frequency=10)

********** now running lsp with #basis dimension = 5 ***********
--------approximate solution path for # itr = 10 complete--------
# epoch: 10	 sup error: 3.808290511369705
--------approximate solution path for # itr = 20 complete--------
# epoch: 20	 sup error: 0.4882354736328125
--------approximate solution path for # itr = 30 complete--------
# epoch: 30	 sup error: 0.3149908185005188
--------approximate solution path for # itr = 40 complete--------
# epoch: 40	 sup error: 0.06118583679199219
--------approximate solution path for # itr = 50 complete--------
# epoch: 50	 sup error: 0.040966540575027466
--------approximate solution path for # itr = 60 complete--------
# epoch: 60	 sup error: 0.02363187074661255
--------approximate solution path for # itr = 70 complete--------
# epoch: 70	 sup error: 0.01431553065776825
--------approximate solution path for # itr = 80 complete--------
# epoch: 80	 sup error: 0.009075149893760681
--------approximate solution path for # itr = 90 complete

In [20]:
def thresh_basis(basis_dim):
    return 0.1**(4)

In [21]:
np.random.seed(8675309)
torch.manual_seed(8675309)

num_itr_history, sup_err_history, breaks = lsp_boosting(input_dim, start_basis_dim, end_basis_dim,
                                        phi_lam, max_epochs, GD_data_loader,  test_data_loader,
                                        loss_fn, lam_min, lam_max, true_losses, init_lr=init_lr,
                                        thresh_basis=thresh_basis, record_frequency=10,
                                        distribution='uniform', device=device, trace_frequency=10)

********** now running lsp with #basis dimension = 5 ***********
--------approximate solution path for # itr = 10 complete--------
# epoch: 10	 sup error: 3.808290511369705
--------approximate solution path for # itr = 20 complete--------
# epoch: 20	 sup error: 0.4882354736328125
--------approximate solution path for # itr = 30 complete--------
# epoch: 30	 sup error: 0.314990758895874
--------approximate solution path for # itr = 40 complete--------
# epoch: 40	 sup error: 0.061185866594314575
--------approximate solution path for # itr = 50 complete--------
# epoch: 50	 sup error: 0.04096655547618866
--------approximate solution path for # itr = 60 complete--------
# epoch: 60	 sup error: 0.023631885647773743
--------approximate solution path for # itr = 70 complete--------
# epoch: 70	 sup error: 0.01431553065776825
--------approximate solution path for # itr = 80 complete--------
# epoch: 80	 sup error: 0.009075149893760681
--------approximate solution path for # itr = 90 complete

In [ ]:
pd.DataFrame(np.array(breaks), columns=['breaks']).to_csv('LSP_results_exact_boosted_breaks_legendre.csv', index=False)

In [ ]:
file_path = 'LSP_results_exact_boosted_legendre.csv'

LSP_results_exact = pd.DataFrame(np.column_stack((num_itr_history, sup_err_history)), columns=['num_itr', 'sup_err'])

# Save the DataFrame to a CSV file
LSP_results_exact.to_csv(file_path, index=False)

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

# Display the DataFrame
df

,num_itr,sup_err
0,10.0,3.808291e+00
1,20.0,4.882355e-01
2,30.0,3.149908e-01
3,40.0,6.118584e-02
4,50.0,4.096654e-02
5,60.0,2.363187e-02
6,70.0,1.431553e-02
7,80.0,9.075150e-03
8,90.0,6.006986e-03
9,100.0,4.100710e-03


# Noisy gradient oracle

In [ ]:
max_epochs = 200

In [ ]:
phi_lam = scaled_shifted_legendre

In [ ]:
def thresh_basis(basis_dim):
    return 0.1**(basis_dim/5 + 5) * (50**2)

In [ ]:
# beta/sqrt(t)
def step_size(t, beta):
    return beta/np.sqrt(t+1)

In [ ]:
start_basis_dim = 5
end_basis_dim = 10
beta = 0.5

In [ ]:
np.random.seed(8675309)
torch.manual_seed(8675309)

num_itr_history, sup_err_history, breaks = lsp_boosting(input_dim, start_basis_dim, end_basis_dim, phi_lam, max_epochs,
                                                        SGD_data_loader, test_data_loader, loss_fn, lam_min, lam_max,
                                                        true_losses, step_size=step_size, const=beta, weighted_avg=True,
                                                        avg_upon_prev=True, thresh_basis=thresh_basis, record_frequency=10,
                                                        distribution='uniform', device=device, trace_frequency=10)

********** now running lsp with #basis dimension = 5 ***********
--------approximate solution path for # itr = 10 complete--------
# epoch: 10	 sup error: 0.11422920227050781
--------approximate solution path for # itr = 20 complete--------
# epoch: 20	 sup error: 0.01976636052131653
--------approximate solution path for # itr = 30 complete--------
# epoch: 30	 sup error: 0.009443342685699463
--------approximate solution path for # itr = 40 complete--------
# epoch: 40	 sup error: 0.002981826663017273
--------approximate solution path for # itr = 50 complete--------
# epoch: 50	 sup error: 0.002051115036010742
--------approximate solution path for # itr = 60 complete--------
# epoch: 60	 sup error: 0.0017272531986236572
--------approximate solution path for # itr = 70 complete--------
# epoch: 70	 sup error: 0.0030344128608703613
--------approximate solution path for # itr = 80 complete--------
# epoch: 80	 sup error: 0.0031889528036117554
--------approximate solution path for # itr = 

In [ ]:
pd.DataFrame(np.array(breaks), columns=['breaks']).to_csv('LSP_results_noisy_boosted_breaks_legendre.csv', index=False)

In [ ]:
file_path = 'LSP_results_noisy_boosted_legendre.csv'

LSP_results_noisy = pd.DataFrame(np.column_stack((num_itr_history, sup_err_history)), columns=['num_itr', 'sup_err'])

# Save the DataFrame to a CSV file
LSP_results_noisy.to_csv(file_path, index=False)

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

# Display the DataFrame
df

,num_itr,sup_err
0,10.0,0.114229
1,20.0,0.019766
2,30.0,0.009443
3,40.0,0.002982
4,50.0,0.002051
...,...,...
88,890.0,0.000264
89,900.0,0.000271
90,910.0,0.000286
91,920.0,0.000291
